In [1]:
import pandas as pd
from prophet import Prophet
import statsmodels.api as sm
import numpy as np

In [2]:
'''
calender = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/calendar.csv')
sales_eval = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/sales_train_evaluation.csv')
sales_val = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/sales_train_validation.csv')
sample_sub = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/sample_submission.csv')
sell_price = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/sell_prices.csv')'''

In [3]:
#sales_eval.head()

In [5]:
sell_price.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


In [4]:
import pandas as pd

# Load your sales and price data
sales_df = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/sales_train_validation.csv')
price_df = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/sell_prices.csv')

# Step 1: Melt the sales dataframe to convert daily sales columns into rows
# sales_df: columns ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd_1', 'd_2', ...]
sales_melted = sales_df.melt(id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], 
                             var_name='d', value_name='sales')

# Step 2: Create a mapping for day `d_X` to actual calendar date (to match with price data)
calendar_df = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/calendar.csv')  # M5 calendar file has the mapping for `d_X` to date and wm_yr_wk

# Merge the sales data with the calendar data to get the `wm_yr_wk` (week) and `date`
sales_with_calendar = pd.merge(sales_melted, calendar_df[['d', 'wm_yr_wk', 'date']], on='d', how='left')

# Step 3: Merge the resulting sales data with the price data using `item_id`, `store_id`, and `wm_yr_wk`
merged_df = pd.merge(sales_with_calendar, price_df, on=['store_id', 'item_id', 'wm_yr_wk'], how='left')

# At this point, merged_df has the columns ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 
# 'd', 'sales', 'wm_yr_wk', 'date', 'sell_price']

# Now you can proceed with the Prophet or linear model
print(merged_df.head())

                              id        item_id    dept_id   cat_id store_id  \
0  HOBBIES_1_001_CA_1_validation  HOBBIES_1_001  HOBBIES_1  HOBBIES     CA_1   
1  HOBBIES_1_002_CA_1_validation  HOBBIES_1_002  HOBBIES_1  HOBBIES     CA_1   
2  HOBBIES_1_003_CA_1_validation  HOBBIES_1_003  HOBBIES_1  HOBBIES     CA_1   
3  HOBBIES_1_004_CA_1_validation  HOBBIES_1_004  HOBBIES_1  HOBBIES     CA_1   
4  HOBBIES_1_005_CA_1_validation  HOBBIES_1_005  HOBBIES_1  HOBBIES     CA_1   

  state_id    d  sales  wm_yr_wk        date  sell_price  
0       CA  d_1      0     11101  2011-01-29         NaN  
1       CA  d_1      0     11101  2011-01-29         NaN  
2       CA  d_1      0     11101  2011-01-29         NaN  
3       CA  d_1      0     11101  2011-01-29         NaN  
4       CA  d_1      0     11101  2011-01-29         NaN  


In [5]:
merged_df.head(20)

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,wm_yr_wk,date,sell_price
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,11101,2011-01-29,NaN
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,11101,2011-01-29,NaN
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,11101,2011-01-29,NaN
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,11101,2011-01-29,NaN
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,11101,2011-01-29,NaN
5,HOBBIES_1_006_CA_1_validation,HOBBIES_1_006,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,11101,2011-01-29,NaN
6,HOBBIES_1_007_CA_1_validation,HOBBIES_1_007,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,11101,2011-01-29,NaN
7,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_1,12,11101,2011-01-29,0.46
8,HOBBIES_1_009_CA_1_validation,HOBBIES_1_009,HOBBIES_1,HOBBIES,CA_1,CA,d_1,2,11101,2011-01-29,1.56
9,HOBBIES_1_010_CA_1_validation,HOBBIES_1_010,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,11101,2011-01-29,3.17


In [6]:
merged_df.tail(20)

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,wm_yr_wk,date,sell_price
58327350,FOODS_3_808_WI_3_validation,FOODS_3_808,FOODS_3,FOODS,WI_3,WI,d_1913,0,11613,2016-04-24,0.94
58327351,FOODS_3_809_WI_3_validation,FOODS_3_809,FOODS_3,FOODS,WI_3,WI,d_1913,0,11613,2016-04-24,1.00
58327352,FOODS_3_810_WI_3_validation,FOODS_3_810,FOODS_3,FOODS,WI_3,WI,d_1913,1,11613,2016-04-24,2.56
58327353,FOODS_3_811_WI_3_validation,FOODS_3_811,FOODS_3,FOODS,WI_3,WI,d_1913,28,11613,2016-04-24,1.88
58327354,FOODS_3_812_WI_3_validation,FOODS_3_812,FOODS_3,FOODS,WI_3,WI,d_1913,3,11613,2016-04-24,2.94
58327355,FOODS_3_813_WI_3_validation,FOODS_3_813,FOODS_3,FOODS,WI_3,WI,d_1913,0,11613,2016-04-24,4.54
58327356,FOODS_3_814_WI_3_validation,FOODS_3_814,FOODS_3,FOODS,WI_3,WI,d_1913,4,11613,2016-04-24,1.88
58327357,FOODS_3_815_WI_3_validation,FOODS_3_815,FOODS_3,FOODS,WI_3,WI,d_1913,0,11613,2016-04-24,2.48
58327358,FOODS_3_816_WI_3_validation,FOODS_3_816,FOODS_3,FOODS,WI_3,WI,d_1913,0,11613,2016-04-24,0.25
58327359,FOODS_3_817_WI_3_validation,FOODS_3_817,FOODS_3,FOODS,WI_3,WI,d_1913,1,11613,2016-04-24,2.94


In [7]:
# Filter the relevant time series for a single store-item combination
# You can loop over all combinations of store_id and item_id
store_id = 'WI_3'  # example store
item_id = 'FOODS_3_820'  # example item

df_filtered = merged_df[(merged_df['store_id'] == store_id) & (merged_df['item_id'] == item_id)].copy()

# Prepare the dataframe for Prophet
df_filtered = df_filtered[['date', 'sales', 'sell_price']]
df_filtered.rename(columns={'date': 'ds', 'sales': 'y', 'sell_price': 'price'}, inplace=True)

In [8]:
df_filtered

,ds,y,price
30482,2011-01-29,1,1.57
60972,2011-01-30,1,1.57
91462,2011-01-31,1,1.57
121952,2011-02-01,1,1.57
152442,2011-02-02,1,1.57
...,...,...,...
58205402,2016-04-20,1,1.98
58235892,2016-04-21,3,1.98
58266382,2016-04-22,6,1.98
58296872,2016-04-23,0,1.98


In [9]:
df_filtered['price'] = df_filtered['price'].fillna(df_filtered['price'].mean())
#df_filtered['price'].fillna(df_filtered['price'].mean(), inplace=True)
df_filtered

,ds,y,price
30482,2011-01-29,1,1.57
60972,2011-01-30,1,1.57
91462,2011-01-31,1,1.57
121952,2011-02-01,1,1.57
152442,2011-02-02,1,1.57
...,...,...,...
58205402,2016-04-20,1,1.98
58235892,2016-04-21,3,1.98
58266382,2016-04-22,6,1.98
58296872,2016-04-23,0,1.98


In [10]:
# Step 1: Fit Prophet model without covariates
prophet_model = Prophet()
prophet_model.fit(df_filtered[['ds', 'y']])

06:04:02 - cmdstanpy - INFO - Chain [1] start processing
06:04:03 - cmdstanpy - INFO - Chain [1] done processing


In [11]:
# Step 2: Make Prophet forecast
future = prophet_model.make_future_dataframe(periods=28)  # forecast for the next 28 days
prophet_forecast = prophet_model.predict(future)

In [12]:
prophet_forecast

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2011-01-29,1.719110,0.060736,3.737488,1.719110,1.719110,0.194287,0.194287,0.194287,0.381198,0.381198,0.381198,-0.186910,-0.186910,-0.186910,0.0,0.0,0.0,1.913397
1,2011-01-30,1.718739,-0.347338,3.429308,1.718739,1.718739,-0.098867,-0.098867,-0.098867,0.096271,0.096271,0.096271,-0.195138,-0.195138,-0.195138,0.0,0.0,0.0,1.619872
2,2011-01-31,1.718368,-0.502680,3.159995,1.718368,1.718368,-0.357350,-0.357350,-0.357350,-0.156558,-0.156558,-0.156558,-0.200792,-0.200792,-0.200792,0.0,0.0,0.0,1.361018
3,2011-02-01,1.717997,-0.589719,3.182094,1.717997,1.717997,-0.419105,-0.419105,-0.419105,-0.215434,-0.215434,-0.215434,-0.203671,-0.203671,-0.203671,0.0,0.0,0.0,1.298892
4,2011-02-02,1.717625,-0.331265,3.221978,1.717625,1.717625,-0.298387,-0.298387,-0.298387,-0.094768,-0.094768,-0.094768,-0.203619,-0.203619,-0.203619,0.0,0.0,0.0,1.419238
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1936,2016-05-18,1.775891,-0.277983,3.541171,1.775811,1.775993,-0.204394,-0.204394,-0.204394,-0.094768,-0.094768,-0.094768,-0.109625,-0.109625,-0.109625,0.0,0.0,0.0,1.571497
1937,2016-05-19,1.776357,-0.345360,3.479811,1.776262,1.776481,-0.224235,-0.224235,-0.224235,-0.098670,-0.098670,-0.098670,-0.125565,-0.125565,-0.125565,0.0,0.0,0.0,1.552122
1938,2016-05-20,1.776823,-0.075822,3.471733,1.776708,1.776974,-0.053875,-0.053875,-0.053875,0.087962,0.087962,0.087962,-0.141837,-0.141837,-0.141837,0.0,0.0,0.0,1.722948
1939,2016-05-21,1.777290,0.016480,3.794866,1.777161,1.777465,0.222957,0.222957,0.222957,0.381198,0.381198,0.381198,-0.158241,-0.158241,-0.158241,0.0,0.0,0.0,2.000247


In [13]:
# Step 3: Calculate residuals (y - Prophet's forecast)
df_filtered['prophet_forecast'] = prophet_forecast['yhat'][:len(df_filtered)]  # Align the forecast with history
df_filtered['residuals'] = df_filtered['y'] - df_filtered['prophet_forecast']

# Step 4: Fit a linear model on the residuals using price as an external regressor
X = sm.add_constant(df_filtered['price'])  # add constant for OLS
linear_model = sm.OLS(df_filtered['residuals'], X).fit()

# Step 5: Predict residuals using the linear model
df_filtered['linear_residuals'] = linear_model.predict(X)

'''
# Step 6: Combine the Prophet forecast and linear model forecast
df_filtered['final_forecast'] = df_filtered['prophet_forecast'] + df_filtered['linear_residuals']

# Step 7: Forecast future residuals and prices for the next 28 days
# Extract future prices for the next 28 days
future_prices = merged_df[(merged_df['store_id'] == store_id) & (merged_df['item_id'] == item_id)].iloc[-28:]['sell_price'].values

# Check for NaN values
if np.any(np.isnan(future_prices)):
    print("Warning: NaN values found in future_prices")

# Add a constant to the future prices
future_X = sm.add_constant(future_prices, has_constant='add')

# Predict future residuals
future_residuals = linear_model.predict(future_X)

# Check if future_residuals contains NaNs
if np.any(np.isnan(future_residuals)):
    print("Warning: NaN values in future_residuals")

# Combine Prophet forecast with linear model forecast
final_forecast = prophet_forecast['yhat'][-28:].values + future_residuals
final_forecast'''

# Step 6: Combine the Prophet forecast and linear model forecast
df_filtered['final_forecast'] = df_filtered['prophet_forecast'].fillna(0) + df_filtered['linear_residuals'].fillna(0)

# Step 7: Forecast future residuals and prices for the next 28 days
future_prices = merged_df[(merged_df['store_id'] == store_id) & (merged_df['item_id'] == item_id)].iloc[-28:]['sell_price'].values

# Handle NaNs in future_prices
future_prices = np.nan_to_num(future_prices, nan=df_filtered['price'].mean())

# Add a constant to the future prices
future_X = sm.add_constant(future_prices, has_constant='add')

# Predict future residuals
future_residuals = linear_model.predict(future_X)

# Handle NaNs in future_residuals
future_residuals = np.nan_to_num(future_residuals, nan=0)

# Combine Prophet forecast with linear model forecast
final_forecast = prophet_forecast['yhat'][-28:].values + future_residuals

# Check for NaNs in the final forecast
if np.any(np.isnan(final_forecast)):
    print("NaN values detected in the final forecast")

final_forecast

array([1.78957601, 1.71037419, 1.81209947, 1.79074496, 1.9614507 ,
       2.24026666, 1.94235187, 1.67784373, 1.60843408, 1.71951205,
       1.70674833, 1.88500745, 2.17011767, 1.87707527, 1.61591689,
       1.5482968 , 1.65962867, 1.64566698, 1.8214145 , 2.10288569,
       1.80530311, 1.53896072, 1.46579118, 1.57149721, 1.55212236,
       1.722948  , 2.00024682, 1.69947008])

In [14]:

# Evaluate the performance using RMSE
def rmse(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred) ** 2))

# Example of evaluation on historical data
train_rmse = rmse(df_filtered['y'], df_filtered['final_forecast'])
print(f'Training RMSE: {train_rmse}')

Training RMSE: 2.1514774461412727


In [37]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from prophet import Prophet

# Assume merged_df is the merged DataFrame with sales and price data

# Get unique store and item combinations
store_item_combinations = merged_df[['store_id', 'item_id']].drop_duplicates()

# Initialize dictionary to store forecasts for each combination
forecasts = {}

# Iterate over each unique store-item combination
for _, row in store_item_combinations.iterrows():
    store_id = row['store_id']
    item_id = row['item_id']
    
    # Filter the data for the current store-item combination
    df_filtered = merged_df[(merged_df['store_id'] == store_id) & (merged_df['item_id'] == item_id)].copy()
    
    # Prepare the data for Prophet
    df_filtered = df_filtered[['date', 'sales', 'sell_price']]
    df_filtered.rename(columns={'date': 'ds', 'sales': 'y', 'sell_price': 'price'}, inplace=True)
    
    # Convert 'ds' to datetime
    df_filtered['ds'] = pd.to_datetime(df_filtered['ds'])
    
    # Fill missing values for price
    df_filtered['price'].fillna(df_filtered['price'].mean(), inplace=True)
    
    # Fit Prophet model
    prophet_model = Prophet()
    prophet_model.fit(df_filtered[['ds', 'y']])
    
    # Make Prophet forecast
    future = prophet_model.make_future_dataframe(periods=28)
    prophet_forecast = prophet_model.predict(future)
    
    # Convert 'ds' in prophet_forecast to datetime
    prophet_forecast['ds'] = pd.to_datetime(prophet_forecast['ds'])
    
    # Merge forecasts with original data to calculate residuals
    df_filtered = pd.merge(df_filtered, prophet_forecast[['ds', 'yhat']], on='ds', how='left')
    df_filtered['residuals'] = df_filtered['y'] - df_filtered['yhat']
    
    # Ensure no missing values in residuals
    df_filtered['residuals'].fillna(0, inplace=True)
    
    # Fit linear model on residuals
    X = sm.add_constant(df_filtered['price'])
    
    # Check for NaNs or infinite values in X and residuals
    if np.any(np.isnan(df_filtered['residuals'])) or np.any(np.isinf(df_filtered['residuals'])):
        print(f"Warning: NaN or infinite values found in residuals for store_id={store_id}, item_id={item_id}")
        df_filtered = df_filtered.dropna(subset=['residuals', 'price'])
    
    if np.any(np.isnan(df_filtered['price'])) or np.any(np.isinf(df_filtered['price'])):
        print(f"Warning: NaN or infinite values found in price for store_id={store_id}, item_id={item_id}")
        df_filtered = df_filtered.dropna(subset=['price'])
    
    # Recompute X and residuals after dropping NaNs
    X = sm.add_constant(df_filtered['price'])
    linear_model = sm.OLS(df_filtered['residuals'], X).fit()
    
    # Extract future prices for the next 28 days
    future_prices = merged_df[(merged_df['store_id'] == store_id) & (merged_df['item_id'] == item_id)].iloc[-28:]['sell_price'].values
    
    # Fill NaN values in future prices if necessary
    if np.any(np.isnan(future_prices)):
        print(f"Warning: NaN values found in future_prices for store_id={store_id}, item_id={item_id}")
        future_prices = np.nan_to_num(future_prices, nan=df_filtered['price'].mean())
    
    # Add a constant to the future prices
    future_X = sm.add_constant(future_prices, has_constant='add')
    
    # Predict future residuals
    future_residuals = linear_model.predict(future_X)
    
    # Check if future_residuals contains NaNs
    if np.any(np.isnan(future_residuals)):
        print(f"Warning: NaN values in future_residuals for store_id={store_id}, item_id={item_id}")
    
    # Combine Prophet forecast with linear model forecast
    final_forecast = prophet_forecast['yhat'][-28:].values + future_residuals
    
    # Store the forecast in the dictionary
    forecasts[(store_id, item_id)] = final_forecast

# Print or process the forecasts
for (store_id, item_id), forecast in forecasts.items():
    print(f"Forecast for store_id={store_id}, item_id={item_id}:")
    print(forecast)


/tmp/ipykernel_36/2844966120.py:30: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_filtered['price'].fillna(df_filtered['price'].mean(), inplace=True)
01:21:30 - cmdstanpy - INFO - Chain [1] start processing
01:21:30 - cmdstanpy - INFO - Chain [1] done processing
/tmp/ipykernel_36/2844966120.py:48: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate

KeyboardInterrupt: 